In [82]:
import requests
import re
import json
from csv import writer 
from io import StringIO
from calendar import monthrange


In [7]:
url = "http://old.emmsa.com.pe/emmsa_spv/app/reportes/ajax/rpt06_getgraph.php"
papas = [
    "01|PAPA AMARILLA",
    "02|PAPA BLANCA/VALLE/OTROS",
    "03|PAPA COLOR/VALLE/OTROS",
    "04|PAPA HUAYRO (ROJO-MORO-NEGRO)RUNT/MARH/U",
		"05|IMPORTADA (AMARILLA/BLANCA/COLOR)",
		"06|PAPA HUAMANTANGA",
		"07|PAPA NEGRA ANDINA",
		"08|PAPA TOMASA",
		"09|PAPA PERUANITA (INJERTO)",
		"10|PAPA PERRICHOLI",
		"11|PAPA YUNGAY",
		"12|PAPA CAPIRO",
		"13|PAPA UNICA",
		"14|PAPA LENGUA",
		"15|PAPA LIBERTEÑA",
		"16|PAPA NEGRA VIUDA",
		"17|PAPA CANCHAN",
		"18|PAPA AMARILIS",
		"19|PAPA NATIVA QEQORANI",
		"20|PAPA NATIVA PUCA SHONGO",
		"21|PAPA NATIVA LEONA",
		"22|PAPA NATIVA SANGRE DE TORO",
		"23|PAPA NATIVA NATIVA MIX",
		"24|PAPA NATIVA SANTO DOMINGO",
		"25|PAPA NATIVA HUAYRO MACHO",
		"26|PAPA NATIVA RITI SISA",
		"27|PAPA NATIVA SUMACC SONCCO",
		"28|PAPA NATIVA WENCCOS",
		"29|PAPA MAMA LUCHA",
		"30|PAPA HUEVO DE INDIO",
		"31|PAPA BELLA"
]

In [13]:
def queryString(papa, yearData, month):
    formattedMonth = str(month)
    if month < 10:
        formattedMonth = "0" + formattedMonth
    return (
        "prod=38|PAPA&var={varPapa}&ser=d&ano={year}&mes={month}".format(
            varPapa=papa, year=str(yearData), month=formattedMonth
        )
    )


In [84]:
from types import NoneType

reg = re.compile("(?<=series: ).*(?=]}])")

output = StringIO()
csv_writer = writer(output)

csv_writer.writerow(
    ["producto", "procedencia", "year", "month", "day", "cantidad (TM)"]
)

for papa in papas:
    for year in range(2015, 2023):
        for month in range(1, 13):
            response = requests.post(
                url,
                headers={
                    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
                },
                data=queryString(papa, year, month),
            )
            body = response.text

            try:
                for entry in json.loads(reg.search(body).group(0) + "]}]"):
                    for day in range(len(entry["data"])):
                        csv_writer.writerow(
                            [
                                papa[3:],
                                entry["name"],
                                year,
                                month,
                                day + 1,
                                entry["data"][day],
                            ]
                        )
            except:
                continue


In [85]:
output.seek(0)
with open('test.csv', mode='w') as f:
    for line in output.getvalue().split('\n'):
        f.write(line)

31